# Import packages

In [2]:
from dataclasses import dataclass, field
import json
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# Create class

In [3]:
@dataclass
class Holiday:
    """A class to describe Holidays during the year."""
    
    #Properties
    name : str
    date : datetime
        
    #Decorator adding an alert when Holidays are deleted
    def del_alert(func):
        def inner(self):
            print(f"{self.name} ({self.date}) has been deleted.")
        return inner

    #Overriding dunder methods to ensure proper functionality
    def __gt__(self, other):
        return self.date > other.date
    def __ge__(self, other):
        return self.date >= other.date
    def __eq__(self, other):
        return self.date == other.date
    
    def __str__(self):
        return f"{self.name}: {self.date}"
    
    @del_alert
    def __del__(self):
        del self

# Program

In [4]:
files = []


for i in range(1,5):
    with open(f"holidays202{i}.json") as infile:
        files.append(json.load(infile))

holidays = []

for i in range(len(files)):
    holidays.append([])
    
for i in range(len(files)):
    for k in range(len(files[i]['holidays'])):
        holidays[i].append(Holiday(name = files[i]['holidays'][k]['name'], date = datetime.strptime(files[i]['holidays'][k]['date'], '%m/%d/%y').date()))

#Requires API key from https://openweathermap.org/
key = "7bce04fe6f84ef261092798ddf193946"

url = f"https://api.openweathermap.org/data/2.5/onecall?lat=44.83206288665719&lon=-93.1458461462507&units=imperial&exclude=current,minutely,hourly,alerts&appid={key}"

predict = requests.get(url).text

weather = json.loads(predict)

forecast = weather['daily'][0:7]

In [5]:
for i in range(len(forecast)):
    forecast[i]['dt'] = datetime.utcfromtimestamp(forecast[i]['dt']).isocalendar().weekday

In [15]:
program = True

while program is True:
    print("Main Menu")
    print("===============")
    print("1. Add a Holiday")
    print("2. Remove a Holiday")
    print("3. Save Holiday List")
    print("4. View Holidays")
    print("5. Exit\n")
    
    response = str(input("Select an option. "))
    
    while response not in ["1", "2", "3", "4", "5"]:
        response = input("Please enter a number. ")
        
    if response == "1":
        print()
        hname = input("Holiday name: ")
        
        valid = False
        
        #Ensure date is in correct format
        while valid is False:
            try:
                hdate = datetime.strptime(input("Holiday date (MM/DD/YY, between 2021-2024): "), "%m/%d/%y").date()
                valid = True
            except:
                continue

        if hdate.year == 2021:
            holidays[0].append(Holiday(name = hname, date = hdate))
            print(f'\nSuccess! {hname} ({hdate}) added.\n')
        elif hdate.year == 2022:
            holidays[1].append(Holiday(name = hname, date = hdate))
            print(f'\nSuccess! {hname} ({hdate}) added.\n')
        elif hdate.year == 2023:
            holidays[2].append(Holiday(name = hname, date = hdate))
            print(f'\nSuccess! {hname} ({hdate}) added.\n')
        elif hdate.year == 2024:
            holidays[3].append(Holiday(name = hname, date = hdate))
            print(f'\nSuccess! {hname} ({hdate}) added.\n')
        else: 
            print("Must be between 2021 and 2024!\n")
        
    elif response == "2":
        print()
        
        found = False
        
        hname = str(input("Holiday name: "))
        
        #Ensure date is in correct format
        for lst in holidays:
            for day in lst:
                if day.name == hname:
                    found = True
        
        #Ensure confirmation response is valid
        if found is True:        
            confirm = input(f'{hname} found. Would you like to delete it? Y/N ').upper()

            while confirm not in ["Y", "N"]:
                confirm = input(f'{hname} found. Would you like to delete it? Y/N ').upper()

            if confirm == 'N':
                print()
                continue
            else:
                for i in range(len(holidays)):
                    holidays[i] = [x for x in holidays[i] if x.name != hname]
                    
        else:
            print(f"{hname} not found.")
        print()
            
    elif response == "3":
        print()
        
        #Ensure confirmation format is valid
        confirm = input("Are you sure you want to save your holidays? Y/N ").upper()
        
        while confirm not in ["Y", "N"]:
            confirm = input("Are you sure you want to save your holidays? Y/N ").upper()
            
        if confirm == "N":
            continue
        else: 
            #Lists of dictionaries to hold holiday data
            temp = [{'holidays' : []}, {'holidays' : []}, {'holidays' : []}, {'holidays' : []}]
            
            #Convert objects to lists of dictionaries
            for i in range(len(holidays)):
                for k in range(len(holidays[i])):
                    temp[i]['holidays'].append({
                        'name': holidays[i][k].name,
                        'date': datetime.strftime(holidays[i][k].date, "%m/%d/%y")
                    })
                    
            for i in range(len(temp)):
                with open(f'holidays202{i + 1}-new.json', 'w') as outfile:
                    json.dump(temp[i], outfile)
                    
    elif response == "4":
        print()
        valid = False
        
        while valid is False:
            try:
                response = int(input("\nPlease enter a week (1-52) to view the holidays from that week. Leave blank for current week."))
                if response in range(1,53):
                    valid = True
            except ValueError:
                response = datetime.today().isocalendar()[1]
                valid = True
                confirm = input("Would you like to view this week's weather? Y/N ").upper()
        
                while confirm not in ["Y", "N"]:
                    confirm = input("Would you like to view this week's weather? Y/N ").upper()
            
            if confirm == "Y":
                weektemp = [[], [], [], [], [], [], [], []]
                weekweather = [[], [], [], [], [], [], [], []]

                for i in range(1,8):
                    for cast in forecast:
                        if cast['dt'] == i:
                            weektemp[i] = cast['temp']['day']

                for i in range(1,8):
                    for cast in forecast:
                        if cast['dt'] == i:
                            weekweather[i] = cast['weather'][0]['description']
            
            print()
                            
            currentyear = int(str(datetime.today().year)[-1])
            output = filter(lambda h: h.date.isocalendar()[1] == response, holidays[currentyear])
            
            if confirm == "Y":
                for o in output:
                    x = o
                    y = weektemp[o.date.isocalendar()[2]]
                    z = weekweather[o.date.isocalendar()[2]]
                    print(f"{x} (Temp: {y}, Weather: {z})")
                    
            print()

        
    elif response == "5":
        print()
        confirm = input("Are you sure you want to exit? Y/N ").upper()
        
        while confirm not in ["Y", "N"]:
            confirm = input("Are you sure you want to exit? Y/N ").upper()

        if confirm == "Y":
            program = False

Main Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Exit

Select an option. 4


Please enter a week (1-52) to view the holidays from that week. Leave blank for current week.
Would you like to view this week's weather? Y/N y

Feast of St Francis of Assisi: 2022-10-04 (Temp: 74.23, Weather: scattered clouds)
World Habitat Day: 2022-10-04 (Temp: 74.23, Weather: scattered clouds)
Frances Xavier Cabrini Day: 2022-10-04 (Temp: 74.23, Weather: scattered clouds)
Child Health Day: 2022-10-04 (Temp: 74.23, Weather: scattered clouds)
World Teachers' Day: 2022-10-05 (Temp: 75.11, Weather: clear sky)
World Cerebral Palsy Day: 2022-10-06 (Temp: 72.3, Weather: light rain)
World Post Day: 2022-10-09 (Temp: 60.85, Weather: overcast clouds)
Leif Erikson Day: 2022-10-09 (Temp: 60.85, Weather: overcast clouds)

Main Menu
1. Add a Holiday
2. Remove a Holiday
3. Save Holiday List
4. View Holidays
5. Exit

Select an option. 5

Are you sure you want to exit? Y/N y
